<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study2/blob/main/feqFeacher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 기본 경로 설정
base_dir = '/mnt/data/ship_data'  # 예시 경로입니다. 실제 경로로 수정하세요.
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']

# 데이터 로드 및 전처리 함수 정의
def load_data(base_dir, split):
    X = []
    y = []
    split_dir = os.path.join(base_dir, split)
    for category in categories:
        category_dir = os.path.join(split_dir, category)
        for file in os.listdir(category_dir):
            file_path = os.path.join(category_dir, file)
            if file_path.endswith('.csv'):
                data = pd.read_csv(file_path, header=None, skiprows=1).values  # 첫 번째 행을 제외하고 데이터 로드
                data = pd.to_numeric(data.flatten(), errors='coerce').reshape(-1, 2)  # 여기서 2는 피처의 수입니다.
                data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
                X.append(data[:, 1])  # y 피처만 추출 (두 번째 열)
                y.append(category)
    X = [np.expand_dims(x, axis=-1) for x in X]  # y 피처만 있으므로 마지막 차원만 추가
    return np.array(X), np.array(y)

# 데이터 로드
X_train, y_train = load_data(base_dir, 'train')
X_val, y_val = load_data(base_dir, 'validation')
X_test, y_test = load_data(base_dir, 'test')

# 데이터 차원 변경 (CNN 입력 형식에 맞게)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# 원-핫 인코딩
y_train_categorical = to_categorical(y_train_encoded)
y_val_categorical = to_categorical(y_val_encoded)
y_test_categorical = to_categorical(y_test_encoded)

# 샘플 데이터 출력
print("X_train shape:", X_train.shape)
print("y_train_categorical shape:", y_train_categorical.shape)
print("X_val shape:", X_val.shape)
print("y_val_categorical shape:", y_val_categorical.shape)
print("X_test shape:", X_test.shape)
print("y_test_categorical shape:", y_test_categorical.shape)

# 첫 번째 샘플 데이터와 레이블 출력
print("\nX_train sample data:", X_train[0])
print("\ny_train_categorical sample:", y_train_categorical[0])

# CNN 모델 정의
def create_cnn_model(input_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# 파라미터 설정
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])  # 입력 형식 조정
num_classes = len(categories)

# 모델 생성
model = create_cnn_model(input_shape, num_classes)

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, y_train_categorical, epochs=50, batch_size=32, validation_data=(X_val, y_val_categorical))

# 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f'테스트 손실: {test_loss:.4f}')
print(f'테스트 정확도: {test_accuracy*100:.2f}%')

# 모델 저장
model.save('ship_fault_classification_model.h5')

# 학습 과정 시각화
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

# 혼동 행렬 출력
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test_categorical, axis=1)

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=categories)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()
